### Q1. Getting the embeddings model

In [2]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
user_question = "I just discovered the course. Can I still join it?"
encoded_question = embedding_model.encode(user_question)
print(f"The first value of the resulting vector is {encoded_question[0]:.2f}")

The first value of the resulting vector is 0.08


### Prepare the documents

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
len(documents)

948

In [11]:
filtered_documents = [doc for doc in documents if doc['course'] == "machine-learning-zoomcamp"]
len(filtered_documents)

375

### Q2. Creating the embeddings

In [17]:
from tqdm import tqdm

embeddings = []

for doc in tqdm(filtered_documents):
    question, text = doc['question'], doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))

import numpy as np

X = np.array(embeddings)
             
print(X.shape)

100%|██████████| 375/375 [00:08<00:00, 43.79it/s] 

(375, 768)


### Q3. Search

In [18]:
scores = X.dot(encoded_question)

In [23]:
print(f"The highest score in the results is {sorted(scores)[-1]:.2f}")

The highest score in the results is 0.65


### Vector search



In [45]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
search_engine.search(encoded_question, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine

In [26]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [55]:
def get_hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [56]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        encoded_question = embedding_model.encode(q['question'])
        results = search_function(encoded_question, num_results=5)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return get_hit_rate(relevance_total)

In [57]:
hit_rate = evaluate(ground_truth, search_engine.search)

100%|██████████| 1830/1830 [00:30<00:00, 60.52it/s]


In [59]:
print(f"The hit rate of VectorSearchEngine is {hit_rate:.2f}")

The hit rate of VectorSearchEngine is 0.94


### Q5. Indexing with Elasticsearch



In [63]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [66]:
for i, doc in enumerate(filtered_documents):
    doc["question_text_vector"] = X[i]
    

In [67]:
for doc in tqdm(filtered_documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:01<00:00, 249.44it/s]


In [97]:
def elastic_search_knn(vector, num_results):
    knn = {
        "field": "question_text_vector",
        "query_vector": vector,
        "k": num_results,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query,
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])
        result_docs[-1]['score'] = hit['_score']


    return result_docs

In [102]:
search_res = elastic_search_knn(encoded_question, num_results=5)
search_res

[{'question': 'The course has already started. Can I still join it?',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693',
  'score': 0.8253288},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DT

In [103]:
print(f"ID with the highest score is {sorted(search_res, key=lambda x: x['score'])[-1]['id']}")

ID with the highest score is ee58a693


### Q6. Hit-rate for Elasticsearch

In [85]:
hit_rate_es = evaluate(ground_truth, elastic_search_knn)

100%|██████████| 1830/1830 [00:26<00:00, 68.39it/s]


In [86]:
print(f"Hitrate for our dataset for ElasticSearch is {hit_rate_es:.2f}")

Hitrate for our dataset for ElasticSearch is 0.94
